In [64]:
import os
import eyed3
from typing import List
from pathlib import Path


eyed3.Fr

def read_mp3_tags(music_files:List[Path]):
    mp3_data = []
    for file in music_files:
        audio_file = eyed3.load(file.resolve())
        if audio_file is not None:
            file_info = {
                "File": file.name,
                "Title": audio_file.tag.title,
                "Artist": audio_file.tag.artist,
                "Album": audio_file.tag.album,
                "Track Number": audio_file.tag.track_num[0] if audio_file.tag.track_num else None,
                "Genre": audio_file.tag.genre.name if audio_file.tag.genre else "Unknown"
            }
            # Style is not directly supported by eyed3, but might be stored in user text frames
            style = None
            if audio_file.tag.user_text_frames:
                for frame in audio_file.tag.user_text_frames:
                    if frame.description.lower() == "style":
                        style = frame.text
            file_info["Style"] = style if style else "Unknown"
            mp3_data.append(file_info)
    return mp3_data

import pandas as pd
music_directory = r"E:\__ANDRIS__\LIBRARY\MUSIC"
music_files = [file for file in Path(music_directory).iterdir() if file.name.endswith(".mp3")]
music_files = music_files[:10]
mp3_tags = read_mp3_tags(music_files)

mp3_tags_df = pd.DataFrame(mp3_tags)
display(mp3_tags_df)


Lame tag CRC check failed
Lame tag CRC check failed
Lame tag CRC check failed
Lame tag CRC check failed
Lame tag CRC check failed


,File,Title,Artist,Album,Track Number,Genre,Style
0,01 - Francis Harris - Hems.mp3,Hems,Francis Harris,Minutes of Sleep,1,Electronic,Unknown
1,01 - Salad Days.mp3,Salad Days,Mac DeMarco,Salad Days,1,Rock,Unknown
2,01 Lotus Collage (1979).mp3,Lotus Collage,Laraaji,Cosmic Tape Experiments,1,Electronic,Unknown
3,01 NN Peaks.mp3,NN/Peaks,Kangding Ray,R-N073 - Stabil,1,Electronic,Unknown
4,02 - Blue Boy.mp3,Blue Boy,Mac DeMarco,Salad Days,2,Rock,Unknown
5,02 - Francis Harris - Dangerdream.mp3,Dangerdream,Francis Harris,Minutes Of Sleep,2,Electronic,Unknown
6,02 Interrompu Court.mp3,Interrompu Court,Kangding Ray,R-N073 - Stabil,2,Electronic,Unknown
7,03 - Brother.mp3,Brother,Mac DeMarco,Salad Days,3,Rock,Unknown
8,03 - Francis Harris - Radiofreeze.mp3,Radiofreeze,Francis Harris,Minutes Of Sleep,3,Electronic,Unknown
9,03 I Am Ocean (1981).mp3,I Am Ocean,Laraaji,Cosmic Tape Experiments,3,Electronic,Unknown


In [30]:
import discogs_client


def search_discogs_by_tags(mp3_tags: List[dict]):
    discogs_data = []
    discogs = discogs_client.Client('MyDiscogsApp/1.0', user_token="vTvLtIhgaKzMHBzMaJqnyzAnpRYPHiSBljzlvuMd")
    for tag in mp3_tags:
        title = tag["Title"]
        artist = tag["Artist"]
        results = discogs.search(f"{title} {artist}", type='release')
        if results:
            release = results[0]
            release_info = {
                "File": tag["File"],
                "Release Title": release.title,
                "Artist": release.artists[0].name if release.artists else "Unknown",
                "Year": release.year,
                "Genre": ", ".join(release.genres) if release.genres else "Unknown",
                "Style": ", ".join(release.styles) if release.styles else "Unknown",  # Added to get style information
                "Discogs Link": release.url,  # Added Discogs link to data
                "Discogs Release ID": release.id  # Added Discogs release ID
            }
            discogs_data.append(release_info)
    return discogs_data

discogs_data = search_discogs_by_tags(mp3_tags)


In [31]:
print(discogs_data)

[{'File': '01 - Francis Harris - Hems.mp3', 'Release Title': 'Francis Harris - Minutes Of Sleep', 'Artist': 'Francis Harris', 'Year': 2014, 'Genre': 'Electronic', 'Style': 'Deep House, Downtempo', 'Discogs Link': 'https://www.discogs.com/release/5439748-Francis-Harris-Minutes-Of-Sleep', 'Discogs Release ID': 5439748}, {'File': '01 - Salad Days.mp3', 'Release Title': 'Mac DeMarco - Salad Days', 'Artist': 'Mac DeMarco', 'Year': 2014, 'Genre': 'Rock', 'Style': 'Indie Rock', 'Discogs Link': 'https://www.discogs.com/release/5552990-Mac-DeMarco-Salad-Days', 'Discogs Release ID': 5552990}, {'File': '01 Lotus Collage (1979).mp3', 'Release Title': 'Laraaji - Lotus Collage', 'Artist': 'Laraaji', 'Year': 1978, 'Genre': 'Electronic', 'Style': 'Ambient, New Age', 'Discogs Link': 'https://www.discogs.com/release/23929163-Laraaji-Lotus-Collage', 'Discogs Release ID': 23929163}, {'File': '01 NN Peaks.mp3', 'Release Title': 'Kangding Ray - Stabil', 'Artist': 'Kangding Ray', 'Year': 2006, 'Genre': 'Elec

In [34]:
discogs_data_df = pd.DataFrame(discogs_data)
discogs_data_df=discogs_data_df.set_index("File")
display(discogs_data_df)


,Release Title,Artist,Year,Genre,Style,Discogs Link,Discogs Release ID
File,,,,,,,
01 - Francis Harris - Hems.mp3,Francis Harris - Minutes Of Sleep,Francis Harris,2014,Electronic,"Deep House, Downtempo",https://www.discogs.com/release/5439748-Francis-Harris-Minutes-Of-Sleep,5439748
01 - Salad Days.mp3,Mac DeMarco - Salad Days,Mac DeMarco,2014,Rock,Indie Rock,https://www.discogs.com/release/5552990-Mac-DeMarco-Salad-Days,5552990
01 Lotus Collage (1979).mp3,Laraaji - Lotus Collage,Laraaji,1978,Electronic,"Ambient, New Age",https://www.discogs.com/release/23929163-Laraaji-Lotus-Collage,23929163
01 NN Peaks.mp3,Kangding Ray - Stabil,Kangding Ray,2006,Electronic,"Glitch, Abstract, Downtempo",https://www.discogs.com/release/733512-Kangding-Ray-Stabil,733512
02 - Blue Boy.mp3,Mac DeMarco - Salad Days,Mac DeMarco,2014,Rock,Indie Rock,https://www.discogs.com/release/5552990-Mac-DeMarco-Salad-Days,5552990
02 - Francis Harris - Dangerdream.mp3,Francis Harris - Minutes Of Sleep,Francis Harris,2014,Electronic,"Deep House, Downtempo",https://www.discogs.com/release/5439748-Francis-Harris-Minutes-Of-Sleep,5439748
02 Interrompu Court.mp3,Kangding Ray - Stabil,Kangding Ray,2006,Electronic,"Glitch, Abstract, Downtempo",https://www.discogs.com/release/733512-Kangding-Ray-Stabil,733512
03 - Brother.mp3,Mac Demarco - The Wonderful World Of Mac DeMarco Singles Club #3,Mac Demarco,2014,Rock,"Indie Rock, Lo-Fi",https://www.discogs.com/release/6172192-Mac-Demarco-The-Wonderful-World-Of-Mac-DeMarco-Singles-Club-3,6172192
03 - Francis Harris - Radiofreeze.mp3,Francis Harris - You Can Always Leave,Francis Harris,2013,Electronic,Deep House,https://www.discogs.com/release/5133130-Francis-Harris-You-Can-Always-Leave,5133130


In [71]:
# Check if the style tag already exists
def user_tag_exist(audio_file, tag_description):
    for frame in audio_file.tag.user_text_frames:
        if frame.description == tag_description:
            return True
    return False

def get_user_tag(audio_file, tag_description):
    for frame in audio_file.tag.user_text_frames:
        if frame.description == tag_description:
            return frame
    raise KeyError(f"{tag_description} does not exist")


from eyed3 import mp3

for file_path in music_files:
    audio_file = eyed3.load(file_path.resolve())
    genre = discogs_data_df.loc[file_path.name]["Genre"]
    styles = [style.strip() for style in discogs_data_df.loc[file_path.name]["Style"].split(",")]
    # audio_file.tag.genre = g
    audio_file.tag.genre = genre

    # set style in User Defined Text frame
    try:
        frame = get_user_tag(audio_file, "Style")
        frame.value = styles
    except KeyError:
        audio_file.tag.user_text_frames.set(text=styles, description="Style")

    audio_file.tag.save()



Lame tag CRC check failed


TypeError: __init__(argument text) must be <class 'str'>

'Electronic'